## Импорт

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from functools import lru_cache
from typing import Dict, Optional
import torch
import re
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge 
from bert_score import BERTScorer
import warnings
import logging
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import logging
from typing import Tuple


## Tuned LLAMA

In [ ]:
class ModelSettings:
    """Настройки для модели unsloth/Llama-3.2-1B-Instruct"""
    def __init__(self):
        # Путь к предобученной модели
        self.model_path = "../models/Llama-merged"
        
        # Параметры квантизации
        self.quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
        )
        
        # Шаблон для форматирования диалога
        self.chat_template = """{% for message in messages %}
            <|im_start|>{{ message['role'] }}\n{{ message['content'] }}<|im_end|>
        {% endfor %}
        <|im_start|>assistant\n"""
        
        # Стандартные параметры генерации текста
        self.default_generation_params = {
            'max_new_tokens': 50,
            'temperature': 0.7,
            'top_p': 0.9,
            'repetition_penalty': 1.1,
            'do_sample': True
        }


@lru_cache(maxsize=None)
def load_model_and_tokenizer(settings: ModelSettings):
    # Инициализация токенизатора
    tokenizer1 = AutoTokenizer.from_pretrained(settings.model_path)
    
    # Загрузка модели с квантизацией
    model1 = AutoModelForCausalLM.from_pretrained(
        pretrained_model_name_or_path=settings.model_path,
        quantization_config=settings.quantization_config,
        device_map="auto",  # Автоматическое распределение по устройствам
        torch_dtype=torch.float16
    )
    
    # Настройка шаблона чата
    tokenizer1.chat_template = settings.chat_template
    
    return model1, tokenizer1


def format_chat_prompt(prompt: str, tokenizer1) -> str:
    messages = [{'role': 'user', 'content': prompt}]
    return tokenizer1.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )


def extract_assistant_response(full_response: str) -> str:
    match = re.search(
        pattern=r"<\|im_start\|>assistant\s*(.*?)(<\|im_end\|>|$)",
        string=full_response,
        flags=re.DOTALL
    )
    return match.group(1).strip() if match else full_response


def generate_tuned_llama_response(
    user_prompt: str,
    generation_params: Optional[Dict] = None,
    settings: Optional[ModelSettings] = None
) -> str:
    try:
        current_settings = settings or ModelSettings()
        generation_params = {**current_settings.default_generation_params, **(generation_params or {})}
        
        model, tokenizer = load_model_and_tokenizer(current_settings)
        
        formatted_prompt = format_chat_prompt(user_prompt, tokenizer)
        
        # Исправлено здесь: model1 -> model, tokenizer1 -> tokenizer
        text_generator = pipeline(
            task="text-generation",
            model=model,        # ← Исправлено
            tokenizer=tokenizer, # ← Исправлено
        )
        
        generation_result = text_generator(formatted_prompt, **generation_params)
        full_response = generation_result[0]['generated_text']
        
        return extract_assistant_response(full_response)
    
    except Exception as error:
        return f"Ошибка при генерации ответа: {str(error)}"


# Стандартное использование
response = generate_tuned_llama_response("you really think hes got hepe")
print("Стандартный ответ:", response)

# Кастомизированный пример
custom_settings = ModelSettings()
custom_settings.default_generation_params['temperature'] = 0.9

custom_response = generate_tuned_llama_response(
    user_prompt="Explain quantum computing in simple terms",
    generation_params={'max_new_tokens': 100},
    settings=custom_settings
)
print("\nКастомизированный ответ:", custom_response)

## Vanilla LLAMA

In [ ]:
# Конфигурация
MODEL_NAME = "unsloth/Llama-3.2-1B-Instruct"
CHAT_TEMPLATE = """{% for message in messages %}
<|im_start|>{{ message['role'] }}
{{ message['content'] }}<|im_end|>
{% endfor %}
<|im_start|>assistant
"""

# Глобальные переменные для модели и токенизатора
model2, tokenizer2 = None, None

def load_model_and_tokenizer():
    """Загрузка модели и токенизатора"""
    global model2, tokenizer2  # Явно объявляем использование глобальных переменных
    
    # Инициализация модели
    model2 = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
        ),
        attn_implementation="eager",
        device_map="auto"
    )
    
    # Загрузка токенизатора
    tokenizer2 = AutoTokenizer.from_pretrained(MODEL_NAME)
    
    # Настройка шаблона чата
    tokenizer2.chat_template = CHAT_TEMPLATE
    tokenizer2.pad_token = tokenizer2.eos_token

def generate_vanilla_llama_response(prompt: str) -> str:
    """Генерация ответа с автоматическим извлечением"""
    global model2, tokenizer2
    
    try:
        # Форматирование промпта
        formatted_prompt = tokenizer2.apply_chat_template(
            [{"role": "user", "content": prompt}],
            tokenize=False,
            add_generation_prompt=True
        )
        
        # Генерация
        inputs = tokenizer2.encode(
            formatted_prompt, 
            return_tensors="pt", 
            add_special_tokens=False
        ).to(model2.device)
        
        outputs = model2.generate(
            inputs,
            max_new_tokens=25,
            do_sample=True,
            temperature=0.5
        )
        
        # Декодирование с пропуском специальных токенов
        full_text = tokenizer2.decode(outputs[0], skip_special_tokens=True)
        
        # Очистка от возможных остаточных тегов
        cleaned_response = re.sub(
            r"<\|im_start\|>|<\|im_end\|>|assistant\s*", 
            "", 
            full_text, 
            flags=re.IGNORECASE
        ).strip()
        
        return cleaned_response if cleaned_response else "Пустой ответ"
    
    except Exception as e:
        return f"Ошибка: {str(e)}"

# Инициализация модели при загрузке скрипта
load_model_and_tokenizer()

# Пример использования
q2 = "you really think hes got hepe"
print(f"{q2}:\n{generate_vanilla_llama_response(q2)}")

## TUNED GPT2

In [ ]:
# Настройка логирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# 1. Сначала определяем функцию загрузки модели
def load_model(
    model_path: str = '../trained_model/',
    tokenizer_path: str = '../trained_tokenizer/',
    device: str = None
) -> Tuple[GPT2LMHeadModel, GPT2Tokenizer]:
    """
    Загружает модель и токенизатор с обработкой ошибок
    """
    try:
        device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
        logger.info(f"Загрузка модели на устройство: {device}")

        tokenizer3 = GPT2Tokenizer.from_pretrained(
            tokenizer_path,
            padding_side='left'
        )
        
        if tokenizer3.pad_token is None:
            tokenizer3.pad_token = tokenizer3.eos_token

        model3 = GPT2LMHeadModel.from_pretrained(model_path).to(device)
        model3.eval()
        
        return model3, tokenizer3

    except Exception as e:
        logger.error(f"Ошибка загрузки модели: {str(e)}")
        raise

# 2. Затем определяем основную функцию генерации
def generate_response(
    model3: GPT2LMHeadModel,
    tokenizer3: GPT2Tokenizer,
    context: str,
    sep_token: str = "<|sep|>",
    max_new_tokens: int = 50,
    temperature: float = 0.7,
    top_k: int = 50,
    top_p: float = 0.9,
    repetition_penalty: float = 1.2,
) -> str:
    """
    Основная функция генерации ответа
    """
    try:
        if not context.strip():
            return "Ошибка: Пустой контекст"
            
        prompt = f"{context.strip()} {sep_token}"
        device = model3.device

        inputs = tokenizer3(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=tokenizer3.model_max_length - max_new_tokens
        ).to(device)

        with torch.no_grad():
            output = model3.generate(
                inputs.input_ids,
                max_new_tokens=max_new_tokens,
                pad_token_id=tokenizer3.eos_token_id,
                do_sample=True,
                temperature=temperature,
                top_k=top_k,
                top_p=top_p,
                repetition_penalty=repetition_penalty,
                no_repeat_ngram_size=2,
                eos_token_id=tokenizer3.eos_token_id,
                early_stopping=True
            )

        full_text = tokenizer3.decode(output[0], skip_special_tokens=False)
        
        if sep_token in full_text:
            response = full_text.split(sep_token)[1]
        else:
            response = full_text[len(tokenizer3.decode(inputs.input_ids[0])):]

        response = response.split(tokenizer3.eos_token)[0].replace("<|endoftext|>", "").strip()
        return ' '.join(response.split()) if response else "Пустой ответ"

    except Exception as e:
        logger.error(f"Ошибка генерации: {str(e)}")
        return "Извините, произошла внутренняя ошибка"

# 3. Затем определяем обертку с загрузкой модели
def generate_tuned_gpt2_response(
    context: str,
    model_path: str = '../trained_model/',
    tokenizer_path: str = '../trained_tokenizer/',
    **kwargs
) -> str:
    """
    Полный цикл генерации с автоматической загрузкой модели
    """
    try:
        model3, tokenizer3 = load_model(model_path, tokenizer_path)
        return generate_response(model3, tokenizer3, context, **kwargs)
    except Exception as e:
        logger.error(f"Ошибка в процессе генерации: {str(e)}")
        return "Извините, произошла внутренняя ошибка"



q3 = "Как настроить VPN на Android?"
print(f"{q3}:\n{generate_tuned_gpt2_response(q3)}")


## Vanilla GPT2

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import logging

# Настройка логирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def generate_vanilla_gpt2_response(
    context: str,
    model_name: str = "gpt2",
    max_new_tokens: int = 50,
    temperature: float = 0.9,
    top_k: int = 50,
    top_p: float = 0.95,
) -> str:
    """
    Генерация ответа с использованием оригинальной GPT-2 модели
    
    Параметры:
        context: Входной контекст/вопрос
        model_name: Название стандартной модели (gpt2, gpt2-medium и т.д.)
        max_new_tokens: Максимальная длина ответа
        temperature: Параметр "творчества" (0.1-1.5)
        top_k: Ограничение словаря для top-k выборки
        top_p: Фильтрация ядерной выборки
        
    Возвращает:
        Сгенерированный ответ
    """
    try:
        # Определение устройства
        device = "cuda" if torch.cuda.is_available() else "cpu"
        
        # Загрузка стандартной модели и токенизатора
        tokenizer4 = GPT2Tokenizer.from_pretrained(model_name)
        model4 = GPT2LMHeadModel.from_pretrained(model_name).to(device)
        model4.eval()
        
        # Подготовка ввода
        inputs = tokenizer4.encode(
            context + "\nresponse:",
            return_tensors="pt",
            max_length=1024,
            truncation=True
        ).to(device)
        
        # Генерация ответа
        with torch.no_grad():
            output = model4.generate(
                inputs,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                top_k=top_k,
                top_p=top_p,
                repetition_penalty=1.2,
                pad_token_id=tokenizer4.eos_token_id,
                do_sample=True,
            )
        
        # Декодирование и постобработка
        full_text = tokenizer4.decode(output[0], skip_special_tokens=True)
        response = full_text[len(tokenizer4.decode(inputs[0])):].strip()
        
        return response.split("\n")[0]  # Берем первую строку ответа

    except Exception as e:
        logger.error(f"Ошибка генерации: {str(e)}")
        return "Не удалось получить ответ"



q4 = "Как настроить VPN на Android?"
print(f"{q4}:\n{generate_vanilla_gpt2_response(q4)}")


## Оценка моделей

In [ ]:
prompt = "What a lovely day today!"
print(generate_tuned_llama_response(prompt))
print(generate_vanilla_llama_response(prompt))
print(generate_tuned_gpt2_response(prompt))
print(generate_vanilla_lgpt2_response(prompt))

In [ ]:
# Импорт необходимых библиотек
import logging
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from rouge import Rouge
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from bert_score import BERTScorer
import matplotlib.patches as mpatches

# Настройка визуализации
plt.style.use('ggplot')
sns.set_palette("husl")
COLORS = {'BLEU': '#3498db', 'ROUGE-L': '#e74c3c', 'BERTScore': '#2ecc71'}

# Конфигурация логирования
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
warnings.filterwarnings('ignore', category=UserWarning)

def load_and_prepare_data(file_path: str, sample_size: int = 10) -> pd.DataFrame:
    """
    Загрузка и подготовка данных
    """
    try:
        df = pd.read_csv(file_path)
        logging.info(f"Успешно загружено {len(df)} записей")
        
        # Очистка данных
        df = df.dropna(subset=['context', 'response']).sample(sample_size, random_state=42)
        return df.reset_index(drop=True)
    
    except Exception as e:
        logging.error(f"Ошибка загрузки данных: {e}")
        raise

def initialize_metrics_tools():
    """
    Инициализация инструментов для вычисления метрик
    """
    return {
        'rouge': Rouge(),
        'bleu_smoother': SmoothingFunction().method1,
        'bertscore': BERTScorer(lang='en', model_type='bert-base-uncased')
    }

def calculate_single_metrics(reference: str, hypothesis: str, tools: dict) -> dict:
    """
    Вычисление метрик для одной пары текст-ответ
    """
    metrics = {}
    
    try:  # BLEU Score
        metrics['BLEU'] = sentence_bleu(
            [reference.split()],
            hypothesis.split(),
            smoothing_function=tools['bleu_smoother']
        )
    except:
        metrics['BLEU'] = 0.0

    try:  # ROUGE-L Score
        if len(reference) > 0 and len(hypothesis) > 0:
            metrics['ROUGE-L'] = tools['rouge'].get_scores(hypothesis, reference)[0]['rouge-l']['f']
        else:
            metrics['ROUGE-L'] = 0.0
    except:
        metrics['ROUGE-L'] = 0.0

    try:  # BERTScore
        _, _, metrics['BERTScore'] = tools['bertscore'].score([hypothesis], [reference])
        metrics['BERTScore'] = metrics['BERTScore'].mean().item()
    except:
        metrics['BERTScore'] = 0.0

    return metrics

def main():
    # Конфигурация
    DATA_PATH = '../data/processed/context_answer.csv'
    MODELS = {
        'tuned_llama': lambda x: "This is a fine-tuned Llama response.",
        'vanilla_llama': lambda x: "Vanilla Llama model response here.",
        'tuned_gpt2': lambda x: "GPT-2 fine-tuned answer generated.",
        'vanilla_gpt2': lambda x: "Basic GPT-2 model response."
    }
    
    # 1. Загрузка данных
    df = load_and_prepare_data(DATA_PATH)
    
    # 2. Инициализация инструментов
    metrics_tools = initialize_metrics_tools()
    
    # 3. Вычисление метрик
    def process_row(row):
        results = {}
        context = row['context']
        reference = row['response']
        
        for model_name, model_fn in MODELS.items():
            hypothesis = model_fn(context)
            results[model_name] = calculate_single_metrics(reference, hypothesis, metrics_tools)
            
        return results
    
    df['metrics'] = df.apply(process_row, axis=1)
    
    # 4. Агрегация результатов
    aggregated = {
        model: {
            metric: np.mean([x[model][metric] for x in df['metrics']])
            for metric in ['BLEU', 'ROUGE-L', 'BERTScore']
        }
        for model in MODELS
    }
    
    # 5. Визуализация
    plot_comparison(aggregated)
    plot_correlation_heatmap(aggregated)
    plot_radar_chart(aggregated)

def plot_comparison(aggregated: dict):
    """
    Основной график сравнения моделей с сохранением стиля
    """
    plt.figure(figsize=(16, 8))
    
    # Подготовка данных
    df_metrics = pd.DataFrame(aggregated).T.reset_index().rename(columns={'index': 'model'})
    sorted_metrics = {metric: df_metrics.sort_values(metric, ascending=False) 
                    for metric in ['BLEU', 'ROUGE-L', 'BERTScore']}
    
    # Построение графиков
    for metric, color in COLORS.items():
        models = sorted_metrics[metric]['model'].tolist()
        scores = sorted_metrics[metric][metric].values
        
        # Основная линия
        plt.plot(np.arange(1, len(models)+1), scores,
                color=color,
                linewidth=2.5,
                marker='o',
                markersize=8,
                label=metric)
        
        # Аннотации
        max_score = max(scores)
        plt.annotate(f'{max_score:.2f}',
                    xy=(1, max_score),
                    xytext=(3, max_score+0.02),
                    color=color,
                    arrowprops=dict(arrowstyle="->", color=color))
    
    # Настройка оформления
    plt.title('Сравнение моделей по метрикам качества', fontsize=14, pad=20)
    plt.xlabel('Ранк модели', fontsize=12)
    plt.ylabel('Нормализованная оценка', fontsize=12)
    plt.xticks(np.arange(1, len(MODELS)+1))
    plt.ylim(0, 1.05)
    plt.grid(True, alpha=0.2, linestyle='--')
    
    # Дополнительные элементы
    add_metric_panel(df_metrics)
    plt.legend(loc='upper right', frameon=True, facecolor='#f5f5f5')
    plt.tight_layout()
    plt.show()

def add_metric_panel(df_metrics: pd.DataFrame):
    """
    Добавление текстовой панели с лучшими моделями
    """
    ax = plt.gca().inset_axes([0.72, 0.68, 0.25, 0.25])
    best_models = {
        metric: df_metrics.loc[df_metrics[metric].idxmax(), 'model']
        for metric in COLORS
    }
    
    text_lines = ["Лучшие модели:"] + [f"{metric}: {model}" 
                                      for metric, model in best_models.items()]
    ax.text(0.1, 0.8, "\n".join(text_lines), fontsize=9)
    ax.axis('off')

def plot_correlation_heatmap(aggregated: dict):
    """
    Heatmap корреляции между метриками
    """
    df = pd.DataFrame(aggregated).T
    corr = df.corr()
    
    plt.figure(figsize=(10, 6))
    sns.heatmap(corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1, fmt=".2f")
    plt.title('Корреляция между метриками оценки', fontsize=14)
    plt.tight_layout()
    plt.show()

def plot_radar_chart(aggregated: dict):
    """
    Радар-чарт для сравнения профилей моделей
    """
    categories = list(COLORS.keys())
    N = len(categories)
    
    angles = np.linspace(0, 2 * np.pi, N, endpoint=False).tolist()
    angles += angles[:1]
    
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, polar=True)
    
    # Настройка осей
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)
    plt.xticks(angles[:-1], categories)
    
    # Построение для каждой модели
    for model, scores in aggregated.items():
        values = [scores[metric] for metric in categories]
        values += values[:1]
        ax.plot(angles, values, linewidth=1, label=model)
        ax.fill(angles, values, alpha=0.1)
    
    # Настройка оформления
    plt.title('Профиль метрик по моделям', y=1.08, fontsize=14)
    plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
    plt.tight_layout()
    plt.show()
